In [1]:
# Run this code if you get errors with missing requirements
import sys
import numpy as np
!{sys.executable} -m pip install mysql-connector
import pandas as pd
import mysql.connector
import statistics
mydb = mysql.connector.connect(
  host="iiwari-mariadb-server",
  user="root",
  passwd="d41k4Duu",
  database="iiwari_org"
)

In [ ]:
# Check database contents
mycursor = mydb.cursor(dictionary=True)
mycursor.execute("SELECT node_id, timestamp, x,y,z,q FROM SensorData WHERE (node_id = 3200 OR node_id = 45300)  AND timestamp > '2019-08-24 13:05:08.078133+00:00' AND timestamp < '2019-09-29 13:05:08.078133+00:00'  AND x > 0  AND X < 10406 AND y < 5220 AND y > 0 AND   q <100 AND z = 0 ");
#mycursor.execute("SELECT node_id, timestamp, x,y,z,q FROM SensorData WHERE node_id = 3200 AND x > 0  AND X < 10406 AND y < 5220 AND y > 0 AND q < 100 LIMIT 100000");
df = pd.DataFrame(mycursor.fetchall())
df['timestamp'] = df['timestamp'].astype(str)
df['timestamp'] = df['timestamp'].str.slice(2, -7)

df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
df['timestamp'] = pd.Series(df['timestamp']).dt.round("S")
df



In [ ]:
import matplotlib.pyplot as plt
img1 = plt.imread("pic.png")
fig, ax = plt.subplots()
#ax.imshow(img, extent=[0, 10406,  5220,0])
ax.imshow(img1, extent=[-800, 10400, 5400, -200])#ax.imshow(img)
#ax.plot(df.x, df.y , linewidth=2, color='firebrick')
df = df.drop(df[(df.x < 1850.0) & (df.y > 3000.0)].index)
ax.plot(df.x, df.y , marker='o', linestyle='dashed', linewidth=0.2, markersize=3, color='firebrick')
print(len(df.x))


In [ ]:
prev = 0
val = 0
def calc_velocity(time_start, time_end):
    #diff_time = time_start - time_end
    diff_time = np.datetime64(time_start) - np.datetime64(time_end)
    diff_time.item().total_seconds()
    diff_time = diff_time / np.timedelta64(1, 's')
    if(diff_time > 0.1):
        return diff_time
    else:
        return 1
x = 0
for i in range(len(df.x)):
   # print(abs(df.iloc[i+1, 3] - abs(df.iloc[i, 3])))
    #print(abs(df.index[i+1]) - abs(df.index[i]))
    if(i < len(df.x)):
        #print(i , " " , x, " ", len(df.x))
        value1 = calc_velocity(df.iloc[i-x, 1], df.iloc[i-(1+x), 1])
        value2 = int((abs(df.iloc[i-x, 2])) - prev)
        val =  value2 / value1
        if (val > 60 or value2 > 100):
            df.drop([df.index[i-x]], axis = 0, inplace = True)
            prev = abs(df.iloc[i-x, 2])
            x +=1
        else:
            prev = abs(df.iloc[i-x, 2])
x = 0
for i in range(len(df.y)):
   # print(abs(df.iloc[i+1, 3] - abs(df.iloc[i, 3])))
    #print(abs(df.index[i+1]) - abs(df.index[i]))
    if(i < len(df.y)):
        value1 = calc_velocity(df.iloc[i-x, 1], df.iloc[i-(1+x), 1])
        value2 = int((abs(df.iloc[i-x, 3]))-prev)
        val = value2 / value1    
        if (val > 60 or value2 > 100):
            df.drop([df.index[i-x]], axis = 0)
            prev = abs(df.iloc[i-x , 3])
            x +=1
        else:
            prev = abs(df.iloc[i-x, 3])
    #print(prev)
    #if (abs(df.iloc[i+1, 3] - abs(df.iloc[i, 3]))) > 400:
        #print("We int")
        
print(len(df.x))

In [ ]:
import matplotlib.pyplot as plt
img1 = plt.imread("pic.png")
fig, ax = plt.subplots()
#ax.imshow(img, extent=[0, 10406,  5220,0])
ax.imshow(img1, extent=[-800, 10400, 5400, -200])
#ax.imshow(img)
plt.rcParams['figure.figsize'] = 10, 5
#ax.plot(df.x, df.y , linewidth=1, color='firebrick')
ax.plot(df.x, df.y , marker='o', linestyle='dashed', linewidth=0.2, markersize=3, color='firebrick')
print(len(df.x))

In [ ]:
#devx = [df.iloc[i + 1, 2] - df.iloc[i, 2] for i in range(len(df.x)-1)]
devx = []
for i in range(len(df.x)):
    value1 = calc_velocity(df.iloc[i, 1], df.iloc[i-1, 1])
    val = int((abs(df.iloc[i, 2]) - prev)) / value1
    #print(val)
    devx.append(val)
    prev = abs(df.iloc[i, 2])

xmean = np.mean(devx, axis=0)
xsd = np.std(devx, axis=0)
print(xmean)
print(xsd)
i = 0
final_list = [x for x in df.x if (abs(x) > xmean - 0.5 * xsd)]
#final_list = [x for x in final_list if (x < xmean + 0. * xsd)]
for x in final_list:
    if(x < xmean + 6.75 * xsd):
        df.drop([df.index[i]], axis = 0, inplace = True)
        i -= 1
    i += 1
i = 0
#devy = [df.iloc[i + 1, 3] - df.iloc[i, 3] for i in range(len(df.y)-1)]
devy = []
for i in range(len(df.y)):
    value1 = calc_velocity(df.iloc[i, 1], df.iloc[i-1, 1])
    val = int((abs(df.iloc[i, 3]) - prev)) / value1
    #print(val)
    devy.append(val)
    prev = abs(df.iloc[i, 3])
ymean = np.mean(devy, axis=0)
ysd = np.std(devy, axis=0)
print(ymean)
print(ysd)
final_list1 = [y for y in df.y if (abs(y) > ymean - 0.5 * ysd)]
#final_list = [y for y in final_list if (y < ymean + 0.2 * ysd)]
i = 0
print(len(df.y))
for y in final_list1:
    if(y < ymean + 6.75 * ysd):
        df.drop([df.index[i]], axis = 0, inplace = True)
        i -= 1
    i += 1
    

In [ ]:
import matplotlib.pyplot as plt
img1 = plt.imread("pic.png")
fig, ax = plt.subplots()
#ax.imshow(img, extent=[-300, 10406,  5220,0])
ax.imshow(img1, extent=[-800, 10400, 5400, -200])#ax.imshow(img)
#ax.plot(df.x, df.y , linewidth=1, color='firebrick')
ax.plot(df.x, df.y , marker='o', linestyle='dashed', linewidth=0.2, markersize=3, color='firebrick')
print(len(df.x))

In [ ]:
import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter


def myplot(x, y, s, bins=1000):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
    heatmap = gaussian_filter(heatmap, sigma=s)

    extent = [xedges[-1], xedges[0], yedges[-1], yedges[0]]
    return heatmap.T, extent


fig, axs = plt.subplots(2, 2)

# Generate some test data
x = df.x
y = df.y
sigmas = [0, 20, 32, 64]
for ax, s in zip(axs.flatten(), sigmas):
    if s == 0:
        ax.imshow(img1, extent=[-800, 10400, 5400, -200])
        ax.plot(x, y, 'k.', markersize=5)
        ax.set_title("Scatter plot")
    else:
        img, extent = myplot(x, y, s)
        #ax.imshow(img1, extent=[-800, 10400, 5400, -200])#ax.imshow(img)
        ax.imshow(img, extent=extent, origin='upper', cmap=cm.jet)
        ax.set_title("Smoothing with  $\sigma$ = %d" % s)

plt.show()

In [ ]:
import seaborn as sns
ax = sns.kdeplot(x, bw=.15)

In [ ]:
prevx = 0
#resy = [df.iloc[i + 1, 3] - df.iloc[i, 3] for i in range(len(df.y)-1)] 
devx = [df.iloc[i + 1, 3] - df.iloc[i, 3] for i in range(len(df.x)-1)]
#devx = statistics.stdev(df.x)
resx = [df.iloc[i + 1, 2] - df.iloc[i, 2] for i in range(len(df.x)-1)] 
i = 0
for cat,date,x,y,z,q in df.values:
    #print(abs(resx[i-1]))
    if(abs(resx[i-1]) < 1):
        df.drop([df.index[i-1]], axis = 0, inplace = True)
        i -= 1
    i += 1

prevy = 0
resy = [df.iloc[i + 1, 3] - df.iloc[i, 3] for i in range(len(df.y)-1)] 
i = 0

for cat,date,x,y,z,q in df.values:
    if(abs(resy[i-1]) < 1):
        df.drop([df.index[i-1]], axis = 0, inplace = True)
        i -= 1
    i += 1


In [ ]:
xmean = np.mean(df.x, axis=0)
xsd = np.std(df.x, axis=0)
ymean = np.mean(df.y, axis=0)
ysd = np.std(df.y, axis=0)

#final_list = [(int((x for x  in df.x if (x > xmean - 0.21 * xsd))), int((y for y in df.y if (y > ymean - 0.21 * ysd))))]
#final_list = [(x,y) for cat,date,x,y,z,q in df.values: if((x > xmean - 0.21 * xsd) and (y > ymean - 0.21 * ysd))]
final_list = []
for cat,date,x,y,z,q in df.values:
    if((x > xmean - 0.2 * xsd) or (y > ymean - 0.2 * ysd)):
        final_list.append([x,y])
print(len(final_list))
i = 0
for x, y in final_list:
    if((x < mean + 0.2 * sd) and (y < mean + 0.2 * sd)):
        df.drop([df.index[i]], axis = 0, inplace = True)
        i -= 1
    i += 1

In [ ]:
resy = [df.iloc[i + 1, 3] - df.iloc[i, 3] for i in range(len(df.y)-1)] 
#resy = df.y['3']-df.y['2']
#resy = df.iloc[1,3] - df.iloc[0,3]
prevy = 0
for y in resy:
    #print(abs(y-prevy))
    #print(y)
    prevy = y
def cal_average(num):
    sum_num = 0
    for t in num:
        sum_num = sum_num + t           

    avg = sum_num / len(num)
    return avg

print("The average is", cal_average(resy))
